In [1]:
using CSV, DataFrames, Dates

using HTTP, Gumbo, Cascadia

In [2]:
# url = "https://www.data.jma.go.jp/svd/eqev/data/bulletin/data/hypo/"

# filelist =["h1919", "h1951", "h1961","h1967","h1983","h1984","h1985","h1986","h1987","h1988","h1989",
# "h1990","h1991","h1992","h1993","h1994","h1995","h1996","h199701","h199710","h1998","h1999",
# "h2000","h2001","h2003","h2004","h2005","h2006","h2007","h2008","h2009",
# "h2010","h2011","h2012","h2013","h2014","h2015","h2016","h2017","h2018","h2019",
# "h202001","h202002","h202003","h202004" ,"h202005" ,"h202006" ,"h202007" ,"h202008"  ];

# filename = url * filelist[1]

In [3]:
url_to_parse = "https://www.data.jma.go.jp/svd/eqev/data/bulletin/hypo.html"

"https://www.data.jma.go.jp/svd/eqev/data/bulletin/hypo.html"

In [4]:
# Make HTTP GET request to the URL and get the response content
response = HTTP.get(url_to_parse)
content = String(response.body)

# Parse the HTML content using Gumbo
parsed_html = parsehtml(content)

# Select the body
body = parsed_html.root[2];

In [6]:
# Select all links from the body of the HTML page
all_links = eachmatch(Cascadia.Selector("a"), body)

# Filter only links that go to a download, results in HTMLElements
downloadable_links = filter(link -> occursin(".zip", lowercase(link.attributes["href"])) || occursin(".tar", lowercase(link.attributes["href"])), all_links)

# Get the actual links
hrefs = [link.attributes["href"] for link in downloadable_links];

In [8]:
corelink = "https://www.data.jma.go.jp/svd/eqev/data/bulletin/"

"https://www.data.jma.go.jp/svd/eqev/data/bulletin/"

In [10]:
download_url = corelink .* hrefs

50-element Vector{String}:
 "https://www.data.jma.go.jp/svd/eqev/data/bulletin/./data/hypo/h202001.zip"
 "https://www.data.jma.go.jp/svd/eqev/data/bulletin/./data/hypo/h202002.zip"
 "https://www.data.jma.go.jp/svd/eqev/data/bulletin/./data/hypo/h202003.zip"
 "https://www.data.jma.go.jp/svd/eqev/data/bulletin/./data/hypo/h202004.zip"
 "https://www.data.jma.go.jp/svd/eqev/data/bulletin/./data/hypo/h202005.zip"
 "https://www.data.jma.go.jp/svd/eqev/data/bulletin/./data/hypo/h202006.zip"
 "https://www.data.jma.go.jp/svd/eqev/data/bulletin/./data/hypo/h202007.zip"
 "https://www.data.jma.go.jp/svd/eqev/data/bulletin/./data/hypo/h202008.zip"
 "https://www.data.jma.go.jp/svd/eqev/data/bulletin/./data/hypo/h2019.zip"
 "https://www.data.jma.go.jp/svd/eqev/data/bulletin/./data/hypo/h2018.zip"
 ⋮
 "https://www.data.jma.go.jp/svd/eqev/data/bulletin/./data/hypo/h1987.zip"
 "https://www.data.jma.go.jp/svd/eqev/data/bulletin/./data/hypo/h1986.zip"
 "https://www.data.jma.go.jp/svd/eqev/data/bulletin/./

In [13]:
mkpath("./downloads")

"./downloads"

In [14]:
HTTP.download.(download_url[1:2],"./downloads")

┌ Info: Downloading
│   source = https://www.data.jma.go.jp/svd/eqev/data/bulletin/./data/hypo/h202001.zip
│   dest = ./downloads\h202001.zip
│   progress = 1.0
│   time_taken = 0.02 s
│   time_remaining = 0.0 s
│   average_speed = 24.577 MiB/s
│   downloaded = 453.001 KiB
│   remaining = 0 bytes
│   total = 453.001 KiB
└ @ HTTP C:\Users\gabipana\.julia\packages\HTTP\z8l0i\src\download.jl:132
┌ Info: Downloading
│   source = https://www.data.jma.go.jp/svd/eqev/data/bulletin/./data/hypo/h202002.zip
│   dest = ./downloads\h202002.zip
│   progress = 1.0
│   time_taken = 0.02 s
│   time_remaining = 0.0 s
│   average_speed = 28.815 MiB/s
│   downloaded = 472.100 KiB
│   remaining = 0 bytes
│   total = 472.100 KiB
└ @ HTTP C:\Users\gabipana\.julia\packages\HTTP\z8l0i\src\download.jl:132


2-element Vector{String}:
 "./downloads\\h202001.zip"
 "./downloads\\h202002.zip"

In [ ]:
mkpath("./downloads")
for link_component in hrefs
    download_url = corelink * link_component
    HTTP.download(download_url,"./downloads")
end;

In [198]:
using ZipFile

In [203]:
function unzip_files(folder_path::AbstractString, destination_path::AbstractString)
    # Get a list of all the files in the folder
    file_list = readdir(folder_path)

    # Filter the list to only include .zip files
    zip_files = filter(x -> occursin(r"\.zip$", x), file_list)

    # Unzip each .zip file in the folder
    for file in zip_files
        # Construct the path to the .zip file
        zip_path = joinpath(folder_path, file)

        # Open the .zip file
        zf = ZipFile.Reader(zip_path)

        # Loop over each file in the .zip file
        for file_in_zip in zf.files
            # Get the name of the file in the .zip file
            file_name = basename(file_in_zip.name)

            # Construct the path to extract the file to
            extract_path = joinpath(destination_path, file_name)

            # Extract the file to the folder
            write(extract_path, read(file_in_zip, String))
        end

        # Close the .zip file
        close(zf)
    end
end

unzip_files (generic function with 1 method)

In [208]:
folder_path = "./downloads"
destination_path = "./data"
mkpath(destination_path)
unzip_files(folder_path, destination_path)

MethodError: MethodError: no method matching unzip_files(::String, ::String)
Closest candidates are:
  unzip_files(::AbstractString) at d:\projects\seismic-networks-julia\data_collection\japan\japan.ipynb:1